In [2]:
import requests
import json

# Read API key from file
with open('../api/congress_gov.txt', 'r') as f:
    api_key = f.read().strip()

# API request
url = 'https://api.congress.gov/v3/law/119'

params = {
    'api_key': api_key,
    'format': 'json',
    'limit': 1
}

response = requests.get(url, params=params)

# Check and print raw JSON response
if response.status_code == 200:
    data = response.json()
    print(json.dumps(data, indent=2))  # Pretty-print the raw JSON
else:
    print(f"Error: {response.status_code} - {response.text}")


{
  "bills": [
    {
      "congress": 119,
      "latestAction": {
        "actionDate": "2025-06-12",
        "text": "Became Public Law No: 119-18."
      },
      "laws": [
        {
          "number": "119-18",
          "type": "Public Law"
        }
      ],
      "number": "160",
      "originChamber": "Senate",
      "originChamberCode": "S",
      "title": "Aerial Firefighting Enhancement Act of 2025",
      "type": "S",
      "updateDate": "2025-06-17",
      "updateDateIncludingText": "2025-06-17",
      "url": "https://api.congress.gov/v3/bill/119/s/160?format=json"
    }
  ],
  "pagination": {
    "count": 18,
    "next": "https://api.congress.gov/v3/law/119?offset=1&limit=1&format=json"
  },
  "request": {
    "congress": "119",
    "contentType": "application/json",
    "format": "json"
  }
}


In [4]:
import requests
import pandas as pd
import time

# Load API key
with open('../api/congress_gov.txt', 'r') as f:
    api_key = f.read().strip()

# Base API endpoint for laws from the 118th Congress
url = 'https://api.congress.gov/v3/law/119'
limit = 250
offset = 0

all_records = []

while True:
    params = {
        'api_key': api_key,
        'format': 'json',
        'limit': limit,
        'offset': offset
    }

    response = requests.get(url, params=params)
    if response.status_code != 200:
        print(f"Error at offset {offset}: {response.status_code}")
        break

    data = response.json()
    bills = data.get("bills", [])
    if not bills:
        break

    for bill in bills:
        # Some bills may have multiple laws attached
        laws = bill.get("laws", [])
        law_numbers = [law.get("number") for law in laws]
        law_types = [law.get("type") for law in laws]

        record = {
            "congress": bill.get("congress"),
            "bill_number": bill.get("number"),
            "bill_type": bill.get("type"),
            "title": bill.get("title"),
            "originChamber": bill.get("originChamber"),
            "latestActionDate": bill.get("latestAction", {}).get("actionDate"),
            "latestActionText": bill.get("latestAction", {}).get("text"),
            "updateDate": bill.get("updateDate"),
            "bill_url": bill.get("url"),
            "law_numbers": "; ".join(law_numbers),
            "law_types": "; ".join(law_types)
        }
        all_records.append(record)

    offset += limit
    print(f"Fetched {offset} laws...")
    time.sleep(0.1)

# Create final DataFrame
df = pd.DataFrame(all_records)
print(f"\n✅ Done! Total laws fetched: {len(df)}")


Fetched 250 laws...

✅ Done! Total laws fetched: 18


In [ ]:
df.